In [1]:
import pandas as pd
import csv
import os
import re
from data_preprocessing import DataPreprocessing

In [2]:
files = os.listdir("3_reviews_per_movie_cleaned")

for file in files:
#     if (file != "The Dark Knight 2008.csv"): continue #uncomment line to test only 1 movie
    csv_file = pd.read_csv("3_reviews_per_movie_cleaned/" + file)
    review_raw = csv_file['review']

In [3]:
sample = 'Confidently directed, dark, brooding, http://www.dhfsj.com and packed with impressive action sequences and a complex story, The Dark Knight includes a career-defining turn from Heath Ledger as well as other Oscar worthy performances, TDK remains not only the best Batman movie, but comic book movie ever created.'
data = DataPreprocessing(sample)
print (data.preprocess())

confidently directed dark brooding packed impressive action sequence complex story dark knight includes career defining turn heath ledger oscar worthy performance tdk remains batman comic book created


In [4]:
csv_file.head()

,review
0,andrew dominiks assassination jesse james cowa...
1,watched trailer hour forty minute long disappo...
2,jesse james newspaper account exploit dime nov...
3,wow style assassination jesse jamed coward rob...
4,writer director andrew dominik long titled len...


In [5]:
csv.register_dialect('myDialect',delimiter=',', quoting=csv.QUOTE_ALL,skipinitialspace=True)
files = os.listdir("3_reviews_per_movie_cleaned")

if not os.path.exists('4_reviews_dataset'):
    os.makedirs('4_reviews_dataset')

complete = 0
percent_complete = 10
num_files = len(files)
for file in files:
    csv_file = pd.read_csv("3_reviews_per_movie_cleaned/" + file)
    review_raw = csv_file['review']
    reviews_selected = []
#     if (file != "The Dark Knight 2008.csv"): continue #uncomment line to test only 1 movie
    cnt = 0
    for review in review_raw:
        if len(review.split()) >= 100:
            reviews_selected.append(review)
            cnt += 1
            if cnt >= 20:
                break
    if len(reviews_selected) < 20:
        complete += 1
        print (file)
        continue
    with open("4_reviews_dataset/" + file, "w") as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['review'])
        for review in reviews_selected:
            review_line = []
            review_line.append(review)
            writer.writerow(review_line)
    complete += 1
    
    if (int(percent_complete*num_files/100) == complete):
        print (f"{percent_complete} % movies cleaned !!")
        percent_complete += 10

10 % movies cleaned !!
20 % movies cleaned !!
Hardball 2001.csv
30 % movies cleaned !!
40 % movies cleaned !!
Step Up 2: The Streets 2008.csv
50 % movies cleaned !!
The Mountain II 2016.csv
The Ottoman Lieutenant 2017.csv
60 % movies cleaned !!
70 % movies cleaned !!
80 % movies cleaned !!
90 % movies cleaned !!
100 % movies cleaned !!


In [6]:
csv.register_dialect('myDialect',delimiter=',', quoting=csv.QUOTE_ALL,skipinitialspace=True)
files = os.listdir("4_reviews_dataset")

if not os.path.exists('5_test_train_dataset'):
    os.makedirs('5_test_train_dataset')
    
genre_list = [
    'Action',
    'Adventure',
    'Animation',
    'Biography',
    'Comedy',
    'Crime',
    'Drama',
    'Fantasy',
    'History',
    'Horror',
    'Music',
    'Mystery',
    'Romance',
    'Sci-Fi',
    'Sport',
    'Thriller',
    'War',
]

rejected_genres = [
    'Superhero',
    'Short',
    'Musical',
    'Film-Noir',
    'Documentary',
    'Family',
    'Western'
]

genre_map = {}

for id, genre in enumerate(genre_list):
    genre_map[genre] = id

files = os.listdir("1_movies_per_genre")
cols_list = ['name', 'genres']
name_to_genre_list = {}
for file in files:
    with open("1_movies_per_genre/" + file, "r") as f:
#         csv_file = pd.read_csv("1_movies_per_genre/" + file, usecols=cols_list)
        csv_file = pd.read_csv("1_movies_per_genre/" + file)
        for index, movie_row in csv_file.iterrows():
            if name_to_genre_list.get(movie_row['name']) is None:
                movie_genres = movie_row['genres'].split(';')
                final_movie_genres = []
                for movie_genre in movie_genres:
                    mg = movie_genre.strip()
                    if mg != '' and mg not in rejected_genres:
                        final_movie_genres.append(mg)
                name_to_genre_list[movie_row['name'] + ' ' + str(movie_row['year'])] = final_movie_genres
                
files = os.listdir("4_reviews_dataset")
complete = 0
percent_complete = 10
num_files = len(files)
headers_x = ['review']
headers_y = []
for genre in genre_list:
    headers_y.append(genre)
# print (headers_y)
with open("5_test_train_dataset/x_test.csv", "w") as x_test:
    with open("5_test_train_dataset/x_train.csv", "w") as x_train:
        with open("5_test_train_dataset/y_test.csv", "w") as y_test:
            with open("5_test_train_dataset/y_train.csv", "w") as y_train:
                
                writer_x_test = csv.writer(x_test, dialect='myDialect')
                writer_x_test.writerow(headers_x)
                
                writer_x_train = csv.writer(x_train, dialect='myDialect')
                writer_x_train.writerow(headers_x)
                
                writer_y_test = csv.writer(y_test, dialect='myDialect')
                writer_y_test.writerow(headers_y)
                
                writer_y_train = csv.writer(y_train, dialect='myDialect')
                writer_y_train.writerow(headers_y)
                
                for file in files:
#                     if file != "Schindler's List 1993.csv":
#                         continue
                    count = 1
                    movie_name = file.split('.csv')[0]
                    if name_to_genre_list.get(movie_name) is None:
                        continue
                    genre_encoding = [0] * len(genre_list)
                    for movie_genre in name_to_genre_list[movie_name]:
                        genre_encoding[genre_map[movie_genre]] = 1
                    csv_file = pd.read_csv("4_reviews_dataset/" + file)
                    review_raw = csv_file['review']
                    for review in review_raw:
                        if count <= 16:
#                             print (genre_encoding)
                            writer_x_test.writerow([review])
                            writer_y_test.writerow(genre_encoding)
                        else:
#                             print (genre_encoding)
                            writer_x_train.writerow([review])
                            writer_y_train.writerow(genre_encoding)
                        count += 1
                    complete += 1

                    if (int(percent_complete*num_files/100) == complete):
                        print (f"{percent_complete} % movies cleaned !!")
                        percent_complete += 10

10 % movies cleaned !!
20 % movies cleaned !!
30 % movies cleaned !!
40 % movies cleaned !!
50 % movies cleaned !!
60 % movies cleaned !!
70 % movies cleaned !!
80 % movies cleaned !!
90 % movies cleaned !!
100 % movies cleaned !!


In [7]:
print (name_to_genre_list["Schindler's List 1993"])

['Biography', 'Drama', 'History']
